## Model I/O - LangChain으로 LLM 호출하기

In [ ]:
!pip install langchain openai

In [1]:
import os
# os.environ["OPENAI_API_KEY"] = "your_api_key"

### Chat / LLM 모델​

두 가지 유형의 언어 모델이 있습니다

- ChatModel: 입력으로 메시지 리스트를 받고 메시지를 반환하는 기본 모델
- LLM: 입력으로 문자열을 받고 문자열을 반환하는 기본 모델 `2024년 1월 4일에 지원 종료.`

```python
from langchain_community.chat_models import ChatOpenAI
from langchain.llms import OpenAI  # will be shut off on January 4th, 2024.

chat_model = ChatOpenAI()
llm = OpenAI()

```

### Message

기본 메시지 인터페이스는 BaseMessage에 의해 정의되며, 두 가지 필수 속성이 있습니다

- content: 메시지의 내용입니다. 보통 문자열입니다.
- role: BaseMessage가 오는 엔티티입니다.


LangChain은 다양한 역할을 쉽게 구분할 수 있는 여러 객체를 제공합니다:

- HumanMessage: 사용자/인간으로부터 오는 BaseMessage입니다.
- AIMessage: AI/어시스턴트로부터 오는 BaseMessage입니다.
- SystemMessage: 시스템으로부터 오는 BaseMessage입니다.
- FunctionMessage / ToolMessage: 함수 또는 도구 호출의 결과를 포함하는 BaseMessage입니다.

```python
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

llm.invoke(text)
# >> Feetful of Fun

chat_model.invoke(messages)
# >> AIMessage(content="Socks O'Color")
```

이들 역할 중 어느 것도 적절하지 않다면, `role`을 수동으로 지정할 수 있는 ChatMessage 클래스도 있습니다.

LangChain은 LLM과 챗모델 모두에 공통적으로 사용되는 인터페이스를 제공합니다.
그러나 주어진 언어 모델에 대한 프롬프트를 가장 효과적으로 구성하기 위해서는 두 모델의 차이점을 이해하는 것이 유용합니다.

LLM이나 챗모델을 호출하는 가장 간단한 방법은 `.invoke()`를 사용하는 것입니다.

- ChatModel.invoke: `BaseMessage의 리스트`를 입력으로 받고 `BaseMessage`를 반환합니다.
- LLM.invoke: 문자열을 입력으로 받고 문자열을 반환합니다.

In [2]:
from langchain_community.chat_models import ChatOpenAI

# 계속 호출 방법이 업데이트 중
# from langchain.chat_models import ChatOpenAI
# from langchain_openai import ChatOpenAI  # pip install langchain_openai 필요

In [3]:
llm = ChatOpenAI()

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_58663/44520985.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [4]:
llm.invoke("안녕")  # 이렇게 호출하는게 권장되진 않음

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 11, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3899fe31-31ca-49f3-95e0-7dee69bbafb1-0')

## Messages

In [5]:
from langchain.schema import SystemMessage, HumanMessage

In [6]:
# 대화 시작
messages = [
    SystemMessage(content="너는 AI 번역 모델이다."),
    HumanMessage(content="'안녕' 이 문장이 영어로 뭐야?"),
]

# 챗 모델을 호출하여 응답을 받습니다.
response = llm.invoke(messages)

In [7]:
response

AIMessage(content="'안녕' 이라는 문장은 영어로 'hello' 또는 'hi'로 번역될 수 있어.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 42, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd500c84-91e5-4c62-b047-71f25b069715-0')

In [8]:
response.content

"'안녕' 이라는 문장은 영어로 'hello' 또는 'hi'로 번역될 수 있어."

## Chain

Prompt template과 LLM 연결하기

In [9]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [10]:
# 요약 template
human_message_prompt = "'{text}' 여기서 키워드를 뽑아서 콤마로 구분해줘"
human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_message_prompt)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt_template])

In [11]:
chain = chat_prompt_template | llm

In [12]:
out = chain.invoke({"text": "LangChain is a framework for developing applications powered by language models."})

In [13]:
out

AIMessage(content='LangChain, framework, developing applications, language models', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 46, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4a1a69ce-fa35-4eb8-aaa8-e0a7e4b1ed5a-0')

In [14]:
out.content

'LangChain, framework, developing applications, language models'